In [1]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>


In [2]:
from math import prod


example = """
    ..##.......
    #...#...#..
    .#....#..#.
    ..#.#...#.#
    .#...##..#.
    ..#.##.....
    .#.#.#....#
    .#........#
    #.##...#...
    #...##....#
    .#..#...#.#
"""


class Toboggan:
    def __init__(self, s: str) -> None:
        self.geology = [l.strip() for l in s.strip().splitlines()]
        self.rows, self.cols = len(self.geology), len(self.geology[0])

    def count_tree_on_path_down(self, d_row: int = 1, d_col: int = 3) -> int:
        row, col = 0, 0
        trees = 0
        while row < self.rows:
            if self.geology[row][col] == "#":
                trees += 1
            row += d_row
            col = (col + d_col) % self.cols

        return trees

    def count_tree_on_path_down_on_slopes(
        self, slopes=((1, 1), (1, 3), (1, 5), (1, 7), (2, 1))
    ) -> int:
        return prod(self.count_tree_on_path_down(d_r, d_c) for d_r, d_c in slopes)

    def __repr__(self) -> str:
        return "\n".join(self.geology)


assert Toboggan(example).count_tree_on_path_down() == 7

In [3]:
with open("../input/day3.txt") as f:
    puzzle = f.read()

print(f"Part I: {Toboggan(puzzle).count_tree_on_path_down()}")

FileNotFoundError: [Errno 2] No such file or directory: '../input/day3.txt'

<link href="style.css" rel="stylesheet"></link>

<main>

<p>Your puzzle answer was <code>268</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Time to check the rest of the slopes - you need to minimize the probability of a sudden arboreal stop, after all.</p>
<p>Determine the number of trees you would encounter if, for each of the following slopes, you start at the top-left corner and traverse the map all the way to the bottom:</p>
<ul>
<li>Right 1, down 1.</li>
<li>Right 3, down 1. (This is the slope you already checked.)</li>
<li>Right 5, down 1.</li>
<li>Right 7, down 1.</li>
<li>Right 1, down 2.</li>
</ul>
<p>In the above example, these slopes would find <code>2</code>, <code>7</code>, <code>3</code>, <code>4</code>, and <code>2</code> tree(s) respectively; multiplied together, these produce the answer <code><em>336</em></code>.</p>
<p><em>What do you get if you multiply together the number of trees encountered on each of the listed slopes?</em></p>
</article>

</main>


In [27]:
assert Toboggan(example).count_tree_on_path_down_on_slopes() == 336

In [28]:
print(f"Part II: { Toboggan(puzzle).count_tree_on_path_down_on_slopes()}")

Part II: 3093068400


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>3093068400</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
